In [1]:
import numpy as np
import pandas as pd

### Считывание данных

In [50]:
hotels = pd.read_csv('task_data/data/hotels_train.csv')

In [51]:
hotels_test = pd.read_csv('task_data/data/hotels_test.csv')

In [52]:
hotels.head()

,hotel_id,stars
0,nxdd9sN7Fi58c_h3iKDyJg,3.5
1,JEs3sXyQu2GlP_aooyymZw,3.5
2,okIjifZcimFjXoXGPJd38w,3.5
3,zsSGQz1EAxrnQ9xMCUCowA,5.0
4,gP_6kJb2JSI3vPjCK2alVA,3.0


In [53]:
#Посмотрим сколько всего у нас классов
labels = hotels.stars.values
np.unique(labels)

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [54]:
len(np.unique(labels))

9

In [55]:
reviews = pd.read_csv('task_data/data/reviews_train.csv')
reviews.head()

,hotel_id,user_id,text
0,awgoBwuhzlLGKEIverTGjQ,yyPHK6khPa4-ErV9U32x3A,Best location ever! I love love love the manag...
1,pR9HHzbUdZ0Gt80_3NmmQw,Yy_iGXxLpL6tYDQoE-6XVg,I'm not good at explaining how I want my hair ...
2,Ex1-tKlQa74J0usZVHxYEw,OQT9DjfBrzrwOEdVJjuYIA,Hands down this place offers the BEST children...
3,Kvr3maTyGSS9Xo_nhsEvrA,FHwl6kaKdD5-KAnfVb_pKQ,Good local shop. I haven't experienced any ra...
4,2Ezp_HYCIVE-h7hpBBvtxw,AlzerMK7z84E4KU6GjPzIQ,I rarely give out 5 stars. It has to be incred...


In [56]:
reviews_test = pd.read_csv('task_data/data/reviews_test.csv')

In [57]:
reviews.tail()

,hotel_id,user_id,text
27789,1OXw5J-nW8se7tPFgyYu3w,ZSjYiQPN2OjA1qg8Hp0_RQ,Fab Fern photo booth was at my wedding in Apri...
27790,K0fhZVoUMY0JaYuMGoMP-A,DE7L3LlEXBTRyYlLLvb1vA,I'd heard about colon hydrotherapy from watchi...
27791,lY2RT75tYZCb2YHWu3bbNQ,7U3H1iJZ04FbadJtt2Npjg,I love the location. It's tucked away and not ...
27792,wXqIh5t-VjN6-jHz6XMXDQ,pJ1spPyqGHDiLR_yTCl-qA,My experience here was absolutely amazing. I r...
27793,LrBCULmDal38G7-UdGM7zA,ER_ZPy4QoitJK9F0qUY0Bw,My own personal service was fine but the recep...


Вообще, вещественные метки классов и требование использовать rmse, как бы тонко намекают, что решаем задачу регресси. Но. Классов у нас всего 9. И их можно преобразовать в целое число, домножением на 10. Поэтому будем решать задачу классификации текста.

Объединим две таблицы: hotels и reviews, по общему полю - hotel_id. Оставив только отзывы (text) и метку класса(stars)


In [111]:
temp_table = pd.merge(hotels, reviews, how='inner', on=['hotel_id'])
temp_table.head()

,hotel_id,stars,user_id,text
0,nxdd9sN7Fi58c_h3iKDyJg,3.5,26roJmjRfsPmA9NDQoVrtQ,I have had them for 3 years now. This year the...
1,nxdd9sN7Fi58c_h3iKDyJg,3.5,JjugfnnQ6g44Ztze8j6F1Q,I have been using Lawn Sense for two summers a...
2,nxdd9sN7Fi58c_h3iKDyJg,3.5,o-gXSZkYNKmYyNBO_e4XGQ,Zach Livingston from Lawn Sense just treated m...
3,JEs3sXyQu2GlP_aooyymZw,3.5,2EuPAGalYnP7eSxPgFCNDg,Spotted this cute food truck while checking ou...
4,JEs3sXyQu2GlP_aooyymZw,3.5,vulPfJ5Q8Hg9nXaKezigfw,I used The Chuck Wagon to cater my events I ha...


In [112]:
dataset = pd.DataFrame(
    {"text":pd.Series(temp_table.text), 
     "stars": pd.Series(temp_table.stars)}
                  )
dataset.head()

,stars,text
0,3.5,I have had them for 3 years now. This year the...
1,3.5,I have been using Lawn Sense for two summers a...
2,3.5,Zach Livingston from Lawn Sense just treated m...
3,3.5,Spotted this cute food truck while checking ou...
4,3.5,I used The Chuck Wagon to cater my events I ha...


In [99]:
dataset.tail()

,stars,text
27789,2.5,"I went yesterday, on a Thursday evening. It wa..."
27790,2.5,This place used to be really dirty -- I mean t...
27791,2.5,Had mixed experiences with this location. Mos...
27792,2.5,The worst looking McDonald's in the whole West...
27793,2.5,Dumpy as they go. I've never consistently wait...


In [61]:
# Запишем полученные данные в csv. Будем пытаться предсказать stars, считая,
# что это метки классов
#df.to_csv(file_name, sep='\t')
dataset.to_csv('Review_stars.csv', sep='\t', index=False)

## Предобработка данных

In [62]:
# Подгрузим стоп-слова, если их не было до этого
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nadezhda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nadezhda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [64]:
from nltk.stem.snowball import RussianStemmer

In [94]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

### Удаление стоп-слов и приведение слов к начальной форме с помощью библиотеки nltk 

In [113]:
# stop_words = set(stopwords.words('english'))
# В дальнейшем хочется использовать n-граммы, поэтому создадим свой словарь стоп-слов(оставим not, no)
stop_words = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
                  'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
                  'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 
                  'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                  'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 
                  'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 
                  'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
                  'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
                  'with', 'about', 'between', 'into', 'through', 'during', 
                  'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 
                  'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
                  'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
                  'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
                  'nor', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 
                  't', 'can', 'will', 'just', 'don', 'should', 'now'])

def preparation_data(data):
    # Разобьем данные на слова
    word_tokens = word_tokenize(data) 
    # Для удаления пунктуации
    words = [word.lower() for word in word_tokens if word.isalpha()]
    # Сделаем стемминг. Приведем слова к начальной форме, используя lemmatizer и стемминг
    wnl = WordNetLemmatizer()
    stemmer = nltk.stem.SnowballStemmer('english')
    words = [stemmer.stem(wnl.lemmatize(word)) for word in words]
    #Чтобы в дальннейшем использовать pipeline и n grams, объединим список слов в строку
    return ' '.join([word for word in words if not word in stop_words])

In [102]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nadezhda\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [114]:
# Перестроим наш датасет
dataset['text'] = dataset['text'].apply(lambda x: preparation_data(x))

In [104]:
dataset.head()

,stars,text
0,3.5,"[year, year, not, done, done, prior, year, wee..."
1,3.5,"[using, lawn, sense, two, summer, could, not, ..."
2,3.5,"[zach, livingston, lawn, sense, treated, paren..."
3,3.5,"[spotted, cute, food, truck, checking, car, su..."
4,3.5,"[used, chuck, wagon, cater, event, held, la, v..."


In [107]:
# Тестовые данные
temp_table = pd.merge(hotels_test, reviews_test, how='inner', on=['hotel_id'])
dataset_test = pd.DataFrame(
    {"hotel_id":pd.Series(temp_table.hotel_id), "text":pd.Series(temp_table.text)}
                  )
dataset_test.head()

,hotel_id,text
0,h7j6MJegP80-mCKamFWVng,Worst limo service ever do not use. Limo was f...
1,h7j6MJegP80-mCKamFWVng,We booked Exotica Limousine Services for our w...
2,h7j6MJegP80-mCKamFWVng,PLEASE READ COMPLETELY BEFORE BOOKING... If yo...
3,h7j6MJegP80-mCKamFWVng,This was the worst experience. First off the d...
4,h7j6MJegP80-mCKamFWVng,We rented a limo for my friend's 30th birthday...


In [108]:
# Произведем предобработку тестовых данных
dataset_test['text'] = dataset_test['text'].apply(lambda x: preparation_data(x))

In [115]:
dataset_test.head()

,hotel_id,text
0,h7j6MJegP80-mCKamFWVng,worst limo service ever not use limo wa filthy...
1,h7j6MJegP80-mCKamFWVng,booked exotica limousine service wedding past ...
2,h7j6MJegP80-mCKamFWVng,please read completely booking looking pay goo...
3,h7j6MJegP80-mCKamFWVng,wa worst experience first driver call repeated...
4,h7j6MJegP80-mCKamFWVng,rented limo friend birthday last night exactly...


In [117]:
np.random.seed(42)

Разделим выборку на тренировочную и валидационную

In [118]:
j = np.random.randint(len(dataset))
valid_dataset = pd.DataFrame(dataset.iloc[[j]])
dataset.drop(dataset.index[j], inplace=True)

In [119]:
# Выберем valid случайно (для воспроизводимости выше зафиксировали random_seed(42))
for i in range(7793):
    j = np.random.randint(len(dataset))
    temp = pd.DataFrame(dataset.iloc[[j]])
    valid_dataset = pd.concat([valid_dataset, temp])
    dataset.drop(dataset.index[j], inplace=True)

In [121]:
len(dataset)

20000

In [122]:
valid_dataset.head()

,stars,text
23654,3,realli love busi cute littl store premium pet ...
15795,4,not use compani ani busi tax advic wa refer vi...
860,2,still understand whi post hour oper anywher ca...
5391,3,servic amic hyper efficac et grand courtoisi u...
21578,2,receiv gift certif birthday onli seem interest...


In [123]:
# Для дальнейшей классификации нам необходимо чтобы метки классов были целыми числами
dataset.stars *= 10
valid_dataset.stars *= 10

In [124]:
dataset.stars.head()

0    35
2    35
3    35
4    35
5    35
Name: stars, dtype: float64

In [125]:
valid_dataset.stars.head()

23654    30
15795    40
860      20
5391     30
21578    20
Name: stars, dtype: float64

### Непосредственно, классификация текста: используется pipeline

In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

### Далее делается векторизация текста по н-граммам (из 2-ух и 3-ех слов) и строится TF-IDF на них

### Попробуем модели: Linear svc, Logistic Regression, Random Forest

In [147]:
from sklearn.svm import LinearSVC

In [151]:
# Используем Linear SVC
pipeline_svc = Pipeline([("vectorizer", CountVectorizer(min_df=3, ngram_range=(2, 3))),
                     ("algo", LinearSVC(random_state=0, tol=1e-5))])
pipeline_svc.fit(X_train, y_train)
print('Train accuracy', accuracy_score(pipeline_svc.predict(X_valid), y_valid))

Train accuracy 0.2560944316140621


In [152]:
pipeline_svc = make_pipeline(CountVectorizer(min_df=3, ngram_range=(2, 3)),
                         TfidfTransformer(),
                         LinearSVC(random_state=0, tol=1e-5))
arr = cross_val_score(pipeline_svc, X_train, y_train, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

[0.26523477 0.27043239 0.26956739 0.25837919 0.26588294]
0.2658993360100569


In [153]:
pipeline_svc.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(2, 3), preprocessor=None, stop_words=None,
  ...hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0))])

In [154]:
print('Train accuracy', accuracy_score(pipeline_svc.predict(X_valid), y_valid))

Train accuracy 0.2850910957146523


In [130]:
dataset.text.values

array(['year year not done done prior year weed galor miss numer area yard pay extra bug control housew get bug like thousand legger millilegg grub paid grub control long yard done',
       'zach livingston lawn sens treat parent grass doe veri grest work friend tell veri prestig job also parent dog reason grass wa alway brown dead befor ever sinc start get treat lawn sens ha never look greener plush ha ever thank take care parent zach lawn sens',
       'spot cute food truck check car super run classic car show want order anyth cumbersom carri sinc want free hand take pix cool car walk wa realli hungri either corn nugget menu would perfect young ladi took order wa friend talk need start tweet sinc never heard said vega streat reli social medium track mani food truck around town wait time wa le minut nugget paper tray veri hot fryer take long cool golden brown nice crunch bit insid creami kernel sweet corn noth spectacular like wa portabl wa messi get place wa happi first order look fo

In [132]:
X_train = list(dataset.text.values)
y_train = list(dataset.stars.valmakeues)
X_valid = list(valid_dataset.text.values)
y_valid = list(valid_dataset.stars.values)

In [ ]:
pipeline = Pipeline([("vectorizer", CountVectorizer(min_df=3, ngram_range=(1, 3))),
                     ("algo", LogisticRegression())])

In [133]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
       ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [198]:
# Before stemming
print('Train accuracy', accuracy_score(pipeline.predict(X_valid), y_valid))

Train accuracy 0.3193403298350825


In [134]:
# After stemming
print('Train accuracy', accuracy_score(pipeline.predict(X_valid), y_valid))

Train accuracy 0.32974082627662304


In [135]:
from sklearn.pipeline import make_pipeline

In [136]:
# After stemming
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 3)),
                         LogisticRegression())
arr = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

[0.27222777 0.27393152 0.28907227 0.27613807 0.27263632]
0.27680118892182115


In [137]:
from sklearn.feature_extraction.text import TfidfTransformer

In [201]:
# Before stemming
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 3)),
                         TfidfTransformer(),
                         LogisticRegression())
arr = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[0.30906274 0.31893044 0.32189257 0.31186967 0.31891368]
0.3161338195099756


In [138]:
# After stemming
pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 3)),
                         TfidfTransformer(),
                         LogisticRegression())
arr = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

[0.31618382 0.31742064 0.31707927 0.31565783 0.3124062 ]
0.3157495525712138


In [139]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [203]:
# Before stemming
print('Train accuracy', accuracy_score(pipeline.predict(X_valid), y_valid))

Train accuracy 0.3423288355822089


In [140]:
# After stemming
print('Train accuracy', accuracy_score(pipeline.predict(X_valid), y_valid))

Train accuracy 0.3418013856812933


Попробуем применить Random Forest

In [141]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [142]:
forest_pipeline = Pipeline([("vectorizer", CountVectorizer(min_df=3, ngram_range=(1, 3))),
                     ("algo", RandomForestClassifier())])

In [143]:
forest_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
       ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [188]:
# Before stemming
print('Train accuracy', accuracy_score(forest_pipeline.predict(X_valid), y_valid))

Train accuracy 0.2843578210894553


In [144]:
# After stemming
print('Train accuracy', accuracy_score(forest_pipeline.predict(X_valid), y_valid))

Train accuracy 0.2734154477803439


In [145]:
# Попробуем изменить параметры
forest_pipeline = Pipeline([("vectorizer", CountVectorizer(min_df=3, ngram_range=(1, 3))),
                     ("algo", RandomForestClassifier(n_estimators=1000))])
forest_pipeline.fit(X_train, y_train)
print('Train accuracy', accuracy_score(forest_pipeline.predict(X_valid), y_valid))

Train accuracy 0.3081857839363613


In [190]:
forest_pipeline = make_pipeline(CountVectorizer(min_df=3, ngram_range=(1, 2)),
                         TfidfTransformer(),
                         RandomForestClassifier())
arr = cross_val_score(forest_pipeline, X_train, y_train, cv=5, scoring='accuracy')
print(arr)
print(np.mean(arr))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

[0.25852053 0.25305173 0.25673841 0.25349108 0.25703201]
0.25576675216095285


In [191]:
forest_pipeline.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
  ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [193]:
print('Train accuracy', accuracy_score(forest_pipeline.predict(X_valid), y_valid))

Train accuracy 0.26486756621689156


Результат линейной регресси чуть более утешающий, посему возьмем его предсказание

In [ ]:
result = pipeline.predict(list(dataset_test.text.values)) / 10

In [ ]:
data_res = pd.DataFrame({'hotel_id': pd.Series(dataset_test.hotel_id), 'stars': pd.Series(result)}, index=None)

In [ ]:
data_res.to_csv('hotels_test.csv', sep=',', index_label=False, index=False)

In [ ]:
from google.colab import files
files.download("hotels_test.csv")